In [1]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/label_for_dev109_test.csv"
#device = "103"
#save_path = f"./data/dev{device}_prepared.csv"
save_path = f"data/label_for_dev109_test_prepared.csv"

df = pd.read_csv(csv_path)
df


#for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng', 'iso_score', 'iso_risk', 'lstm_score', 'lstm_risk', 'confidence', 'rf_risk', 'devID']:

#

,timestamp,device,devID,temp,humi,soil,rain,geo,lat,long,risk,ISO Risk,LSTM Risk,RF Risk,label
0,11/20/2025 9:37,109,Node 109,23.1,99.1,46.6,0.84,0.28,8.868584,99.74958,ต่ำ,0,0,0,NaN
1,11/20/2025 9:36,109,Node 109,23.1,99.1,46.6,0.70,0.45,8.868584,99.74958,ต่ำ,0,0,0,NaN
2,11/20/2025 9:34,109,Node 109,23.1,99.1,46.6,0.56,0.62,8.868584,99.74958,ต่ำ,0,0,0,NaN
3,11/20/2025 9:34,109,Node 109,23.1,99.1,46.6,0.56,0.42,8.868584,99.74958,ต่ำ,0,0,0,NaN
4,11/20/2025 9:33,109,Node 109,23.1,99.1,46.6,0.56,0.42,8.868584,99.74958,ต่ำ,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6569,11/16/2025 11:15,109,Node 109,30.3,68.4,35.4,0.00,0.36,8.868586,99.749557,ต่ำ,0,0,0,NaN
6570,11/16/2025 11:14,109,Node 109,30.1,70.1,35.4,0.00,0.31,8.868586,99.749557,ต่ำ,0,0,0,NaN
6571,11/16/2025 11:14,109,Node 109,30.1,72.9,35.4,0.00,0.33,8.868586,99.749557,ต่ำ,0,0,0,NaN
6572,11/16/2025 11:13,109,Node 109,29.9,70.7,35.4,0.00,0.33,8.868586,99.749557,ต่ำ,0,0,0,NaN


In [2]:
df['devID'] = df['devID'].str.replace('Node ', '', regex=False)

In [3]:
df.drop(columns=['device'], inplace=True)

In [4]:
for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'devID']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

#def clean_and_format_timestamp(val):
#    if pd.isnull(val):
#        return None
#    val = str(val).replace('(ICT)', '').strip()
#    try:
#        dt = pd.to_datetime(val)
#        return dt.isoformat()
#    except Exception:
#        return None  
#
#df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

#df = df.drop(columns=['Unnamed: 15', 'confidence','rf_risk'], errors='ignore')
##df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
#df = df[df['lng'] > 15.0]
df = df[df['soil'] > 0]
df = df[df['rain'] >= 0]
df = df[df['humi'] > 0]
#df = df[(df['devID'] == 104) | (df['devID'] == 103)]
df = df[(df['geo'] >= 0) & (df['geo'] < 40)]
df = df[df['temp'] > 0]
df = df[(df['devID'] > 0) & (df['devID'] < 1000)]
#df = df[df['iso_score'] > -85000]
#df = df[df['lstm_score'] >= 0]
#df = df[df['timestamp'] > '3035-09-11']

#df = df[df['devID'] == int(device)]
df

,timestamp,devID,temp,humi,soil,rain,geo,lat,long,risk,ISO Risk,LSTM Risk,RF Risk,label
0,2025-11-20 09:37:00,109,23.1,99.1,46.6,0.84,0.28,8.868584,99.74958,ต่ำ,0,0,0,NaN
1,2025-11-20 09:36:00,109,23.1,99.1,46.6,0.70,0.45,8.868584,99.74958,ต่ำ,0,0,0,NaN
2,2025-11-20 09:34:00,109,23.1,99.1,46.6,0.56,0.62,8.868584,99.74958,ต่ำ,0,0,0,NaN
3,2025-11-20 09:34:00,109,23.1,99.1,46.6,0.56,0.42,8.868584,99.74958,ต่ำ,0,0,0,NaN
4,2025-11-20 09:33:00,109,23.1,99.1,46.6,0.56,0.42,8.868584,99.74958,ต่ำ,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6569,2025-11-16 11:15:00,109,30.3,68.4,35.4,0.00,0.36,8.868586,99.749557,ต่ำ,0,0,0,NaN
6570,2025-11-16 11:14:00,109,30.1,70.1,35.4,0.00,0.31,8.868586,99.749557,ต่ำ,0,0,0,NaN
6571,2025-11-16 11:14:00,109,30.1,72.9,35.4,0.00,0.33,8.868586,99.749557,ต่ำ,0,0,0,NaN
6572,2025-11-16 11:13:00,109,29.9,70.7,35.4,0.00,0.33,8.868586,99.749557,ต่ำ,0,0,0,NaN


In [5]:
df.describe()

,timestamp,devID,temp,humi,soil,rain,geo,lat,ISO Risk,LSTM Risk,RF Risk
count,6566,6566.0,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000
mean,2025-11-18 09:56:13.259214080,109.0,24.389225,94.961362,42.099269,1.745815,5.493713,8.751685,0.266677,0.185958,0.395218
min,2025-11-16 11:12:00,109.0,22.500000,52.100000,34.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000
25%,2025-11-17 10:21:15,109.0,23.250000,95.650000,35.400000,0.000000,0.380000,8.868585,0.000000,0.000000,0.000000
50%,2025-11-18 10:30:30,109.0,23.800000,98.500000,42.900000,0.000000,0.580000,8.868594,0.000000,0.000000,0.000000
75%,2025-11-19 08:27:45,109.0,24.200000,98.800000,45.600000,0.840000,8.055000,8.868604,0.000000,0.000000,1.000000
max,2025-11-20 09:37:00,109.0,34.200000,99.300000,52.300000,45.680000,32.000000,8.916667,2.000000,2.000000,2.000000
std,NaN,0.0,2.138913,8.855542,5.273759,4.813609,8.344360,1.026293,0.613885,0.548896,0.668678


In [6]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

#for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng', 'iso_risk', 'iso_score', 'lstm_risk']:
for col in ['soil', 'rain', 'temp', 'humi', 'geo']:

    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,devID,temp,humi,soil,rain,geo,lat,long,risk,ISO Risk,LSTM Risk,RF Risk,label
0,2025-11-20 09:37:00,109,23.1,99.1,46.6,0.84,0.28,8.868584,99.74958,ต่ำ,0,0,0,NaN
1,2025-11-20 09:36:00,109,23.1,99.1,46.6,0.70,0.45,8.868584,99.74958,ต่ำ,0,0,0,NaN
2,2025-11-20 09:34:00,109,23.1,99.1,46.6,0.56,0.62,8.868584,99.74958,ต่ำ,0,0,0,NaN
3,2025-11-20 09:34:00,109,23.1,99.1,46.6,0.56,0.42,8.868584,99.74958,ต่ำ,0,0,0,NaN
4,2025-11-20 09:33:00,109,23.1,99.1,46.6,0.56,0.42,8.868584,99.74958,ต่ำ,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6569,2025-11-16 11:15:00,109,30.3,68.4,35.4,0.00,0.36,8.868586,99.749557,ต่ำ,0,0,0,NaN
6570,2025-11-16 11:14:00,109,30.1,70.1,35.4,0.00,0.31,8.868586,99.749557,ต่ำ,0,0,0,NaN
6571,2025-11-16 11:14:00,109,30.1,72.9,35.4,0.00,0.33,8.868586,99.749557,ต่ำ,0,0,0,NaN
6572,2025-11-16 11:13:00,109,29.9,70.7,35.4,0.00,0.33,8.868586,99.749557,ต่ำ,0,0,0,NaN


In [7]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')
#df['hour'] = df.index.hourผ
#df['day_of_week'] = df.index.dayofweek
df = df.reset_index() 

In [17]:
df

,timestamp,devID,temp,humi,soil,rain,geo,lat,long,risk,ISO Risk,LSTM Risk,RF Risk,label
0,2025-11-16 11:12:00,109,29.9,70.7,35.4,0.00,0.33,8.868586,99.749557,ต่ำ,0,0,0,NaN
1,2025-11-16 11:13:00,109,29.9,70.7,35.4,0.00,0.33,8.868586,99.749557,ต่ำ,0,0,0,NaN
2,2025-11-16 11:14:00,109,30.1,70.1,35.4,0.00,0.31,8.868586,99.749557,ต่ำ,0,0,0,NaN
3,2025-11-16 11:14:00,109,30.1,72.9,35.4,0.00,0.33,8.868586,99.749557,ต่ำ,0,0,0,NaN
4,2025-11-16 11:15:00,109,30.3,68.4,35.4,0.00,0.36,8.868586,99.749557,ต่ำ,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6561,2025-11-20 09:33:00,109,23.1,99.1,46.6,0.56,0.42,8.868584,99.74958,ต่ำ,0,0,0,NaN
6562,2025-11-20 09:34:00,109,23.1,99.1,46.6,0.56,0.42,8.868584,99.74958,ต่ำ,0,0,0,NaN
6563,2025-11-20 09:34:00,109,23.1,99.1,46.6,0.56,0.62,8.868584,99.74958,ต่ำ,0,0,0,NaN
6564,2025-11-20 09:36:00,109,23.1,99.1,46.6,0.70,0.45,8.868584,99.74958,ต่ำ,0,0,0,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6566 entries, 0 to 6565
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  6566 non-null   datetime64[ns]
 1   devID      6566 non-null   int64         
 2   temp       6566 non-null   float64       
 3   humi       6566 non-null   float64       
 4   soil       6566 non-null   float64       
 5   rain       6566 non-null   float64       
 6   geo        6566 non-null   float64       
 7   lat        6566 non-null   float64       
 8   long       6566 non-null   object        
 9   risk       6566 non-null   object        
 10  ISO Risk   6566 non-null   int64         
 11  LSTM Risk  6566 non-null   int64         
 12  RF Risk    6566 non-null   int64         
 13  label      1104 non-null   object        
dtypes: datetime64[ns](1), float64(6), int64(4), object(3)
memory usage: 718.3+ KB


In [9]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
label    5462
dtype: int64


In [10]:
#df = df.dropna(subset=['humi']).reset_index(drop=True)

In [11]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
label    5462
dtype: int64


In [12]:
#df = df.dropna()
#
#nan_counts = df.isna().sum()
#nan_columns = nan_counts[nan_counts > 0]
#print("df102")
#print("📌 คอลัมน์ที่มีค่า NaN:")
#print(nan_columns)

In [13]:
#df[(df["iso_risk"]==2) & (df["lstm_risk"]==2)]

In [14]:
df.describe()

,timestamp,devID,temp,humi,soil,rain,geo,lat,ISO Risk,LSTM Risk,RF Risk
count,6566,6566.0,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000
mean,2025-11-18 09:56:13.259214080,109.0,24.389225,94.961362,42.099269,1.745815,5.493713,8.751685,0.266677,0.185958,0.395218
min,2025-11-16 11:12:00,109.0,22.500000,52.100000,34.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000
25%,2025-11-17 10:21:15,109.0,23.250000,95.650000,35.400000,0.000000,0.380000,8.868585,0.000000,0.000000,0.000000
50%,2025-11-18 10:30:30,109.0,23.800000,98.500000,42.900000,0.000000,0.580000,8.868594,0.000000,0.000000,0.000000
75%,2025-11-19 08:27:45,109.0,24.200000,98.800000,45.600000,0.840000,8.055000,8.868604,0.000000,0.000000,1.000000
max,2025-11-20 09:37:00,109.0,34.200000,99.300000,52.300000,45.680000,32.000000,8.916667,2.000000,2.000000,2.000000
std,NaN,0.0,2.138913,8.855542,5.273759,4.813609,8.344360,1.026293,0.613885,0.548896,0.668678


In [15]:
df['devID'].value_counts()

devID
109    6566
Name: count, dtype: int64

In [16]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
